In [64]:
sentences = [  
    # Botanic context  
    "I planted a maple tree in the backyard.",  
    "The tree provides shade during hot summer days.",  
    "We sat under the old oak tree.",  
    "The tree blossoms in the spring.",  
    "The apple tree bears fruit every autumn.",  
    "The tree's roots were deep in the ground.",  
    "The tree's leaves turned yellow and fell off.",  
    "The pine tree stood tall in the forest.",  
    "The willow tree hung over the pond.",  
    "The tree was cut down to make room for new construction.",  
      
    # Computer programming context  
    "The binary tree is a fundamental data structure in computer science.",  
    "Each node in the tree stores a piece of data.",  
    "The tree structure allows efficient search and sort operations.",  
    "The tree is traversed in a pre-order, in-order, or post-order manner.",  
    "A balanced binary tree offers optimal performance.",  
    "The tree's root node has no parent.",  
    "Each node in the tree has a link to its parent and children.",  
    "The tree's leaf nodes have no children.",  
    "A tree in computer science is not necessarily rooted.",  
    "The tree algorithm was implemented recursively.",  
      
    # Family tree context  
    "My family tree traces back to the 16th century.",  
    "I am researching my family tree.",  
    "My family tree has branches all over the world.",  
    "The family tree shows our genealogy.",  
    "I found an interesting ancestor in our family tree.",  
    "My family tree is quite complex.",  
    "Our family tree includes several notable individuals.",  
    "The family tree reveals our heritage.",  
    "I discovered distant relatives through the family tree.",  
    "The family tree helps us understand our roots.",  
]  


In [78]:
import os

# https://www.sbert.net/docs/pretrained_models.html
transformers_cache = os.environ.get('TRANSFORMERS_CACHE')
print(transformers_cache)

from sentence_transformers import SentenceTransformer
# https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder=transformers_cache)
#model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder=transformers_cache)
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1', cache_folder=transformers_cache)
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', cache_folder=transformers_cache)

None


In [79]:
from pprint import pprint
# Now we want to transform all sentences with bert

embeddings = model.encode(sentences)
#and now I want to print a matrix with the cosine similarity of each sentence with each other sentence
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
df = pd.DataFrame(cosine_similarity(embeddings))
df.columns = range(len(sentences))
df.index = range(len(sentences))
#print only the first sentence
df


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,0.478877,0.419012,0.483624,0.287598,0.496032,0.357591,0.381098,0.286593,0.415501,...,0.337012,0.325625,0.410464,0.278653,0.383755,0.360111,0.285614,0.302082,0.300678,0.358039
1,0.478877,1.000000,0.328972,0.618384,0.423258,0.381629,0.413176,0.396305,0.343274,0.400440,...,0.269429,0.265807,0.381334,0.224611,0.242920,0.242419,0.260255,0.277028,0.138424,0.293001
2,0.419012,0.328972,1.000000,0.319364,0.219391,0.444657,0.294564,0.355661,0.284443,0.439110,...,0.355860,0.254866,0.258597,0.321574,0.377359,0.247804,0.254358,0.348362,0.296948,0.309678
3,0.483624,0.618384,0.319364,1.000000,0.447094,0.421011,0.401091,0.429895,0.375166,0.365935,...,0.307519,0.325869,0.405192,0.334758,0.240674,0.286263,0.309548,0.349940,0.127047,0.323464
4,0.287598,0.423258,0.219391,0.447094,1.000000,0.340903,0.271181,0.246399,0.184936,0.245341,...,0.200784,0.236467,0.296921,0.273688,0.229642,0.253107,0.263766,0.289663,0.194460,0.311810
5,0.496032,0.381629,0.444657,0.421011,0.340903,1.000000,0.477540,0.505154,0.297986,0.592978,...,0.354059,0.281131,0.395371,0.319652,0.336156,0.315351,0.247506,0.359109,0.283290,0.558303
6,0.357591,0.413176,0.294564,0.401091,0.271181,0.477540,1.000000,0.405738,0.335222,0.466901,...,0.234018,0.163733,0.263832,0.168761,0.171673,0.183847,0.143013,0.227089,0.111049,0.206230
7,0.381098,0.396305,0.355661,0.429895,0.246399,0.505154,0.405738,1.000000,0.423582,0.474463,...,0.231942,0.203100,0.345279,0.247852,0.212301,0.225366,0.241128,0.239513,0.138207,0.285666
8,0.286593,0.343274,0.284443,0.375166,0.184936,0.297986,0.335222,0.423582,1.000000,0.327690,...,0.191085,0.168785,0.319778,0.194258,0.176992,0.176993,0.164011,0.187810,0.108877,0.160746
9,0.415501,0.400440,0.439110,0.365935,0.245341,0.592978,0.466901,0.474463,0.327690,1.000000,...,0.322161,0.298749,0.387844,0.323635,0.293326,0.322815,0.274401,0.375968,0.211190,0.382669


In [74]:
import numpy as np
import umap
import altair as alt

# declare a function to plot the embeddings
def plot_embeddings(sentences, embeddings):

    panda_sentences = pd.DataFrame({'text':sentences})

    # UMAP reduces dimension to a plottable 2DE
    reducer = umap.UMAP(n_neighbors=2)
    umap_embeds = reducer.fit_transform(embeddings)
    # create a dataframe with the umap embeddings and the corresponding sentences
    df_plot = pd.DataFrame({'x': umap_embeds[:, 0], 'y': umap_embeds[:, 1], 'text': panda_sentences['text']})

    # create the interactive scatter plot with labels
    return alt.Chart(df_plot, width=1100, height=600).mark_circle(size=60).encode(
        x='x',
        y='y',
        tooltip=['text']
    )


In [75]:
# now we can plot the embeddings
chart  = plot_embeddings(sentences, embeddings)

chart.interactive()

/Users/gianmariaricci/develop/github/ai-playground/src/python/langchainVarious/langchain/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [76]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#print out the value of the environment variable OPENAI_API_KEY

import openai

openai.api_type = "azure"
openai.api_key = os.environ.get("OPENAI_API_KEY")   
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = "2023-05-15"

embeddings_ada = []
for sentence in sentences:
    response = openai.Embedding.create(
        input=sentence,
        engine="text-embedding-ada-002"
    )
    embeddings_ada.append(response['data'][0]['embedding'])
    
# Now we can plot the embeddings
chart  = plot_embeddings(sentences, embeddings_ada)
chart.interactive()



/Users/gianmariaricci/develop/github/ai-playground/src/python/langchainVarious/langchain/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [77]:
df = pd.DataFrame(cosine_similarity(embeddings_ada))
df.columns = range(len(sentences))
df.index = range(len(sentences))
#print only the first sentence
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,0.827971,0.843640,0.844273,0.825876,0.830557,0.831997,0.842284,0.844910,0.830044,...,0.810372,0.801517,0.824198,0.803154,0.807074,0.809565,0.794221,0.798882,0.802025,0.790026
1,0.827971,1.000000,0.838071,0.866920,0.846457,0.834051,0.836201,0.849261,0.843288,0.828038,...,0.773922,0.753445,0.793168,0.790201,0.760604,0.776242,0.780977,0.798875,0.762647,0.802966
2,0.843640,0.838071,1.000000,0.807304,0.819562,0.852186,0.834298,0.855319,0.865310,0.826110,...,0.808189,0.782789,0.800651,0.812299,0.807888,0.799571,0.807306,0.811558,0.799291,0.809006
3,0.844273,0.866920,0.807304,1.000000,0.866546,0.836477,0.855233,0.851744,0.853183,0.822889,...,0.783908,0.769665,0.808522,0.808345,0.771259,0.789213,0.785359,0.812013,0.772505,0.803040
4,0.825876,0.846457,0.819562,0.866546,1.000000,0.807877,0.852751,0.822655,0.818802,0.793739,...,0.781963,0.755217,0.797756,0.796574,0.763566,0.776510,0.782380,0.802873,0.757589,0.792241
5,0.830557,0.834051,0.852186,0.836477,0.807877,1.000000,0.832000,0.869080,0.851926,0.833608,...,0.799845,0.768863,0.810203,0.802451,0.794694,0.810042,0.792298,0.804245,0.800030,0.822685
6,0.831997,0.836201,0.834298,0.855233,0.852751,0.832000,1.000000,0.841601,0.846364,0.846524,...,0.757263,0.748312,0.784238,0.774401,0.757561,0.772930,0.761465,0.773459,0.761697,0.765336
7,0.842284,0.849261,0.855319,0.851744,0.822655,0.869080,0.841601,1.000000,0.871237,0.845496,...,0.790575,0.760813,0.808818,0.802327,0.782903,0.790888,0.795707,0.803424,0.783052,0.794334
8,0.844910,0.843288,0.865310,0.853183,0.818802,0.851926,0.846364,0.871237,1.000000,0.834351,...,0.778950,0.756909,0.796629,0.797277,0.774248,0.786217,0.787564,0.797183,0.771715,0.787870
9,0.830044,0.828038,0.826110,0.822889,0.793739,0.833608,0.846524,0.845496,0.834351,1.000000,...,0.758014,0.748048,0.767949,0.774289,0.755881,0.765223,0.764060,0.774019,0.749549,0.770475
